## Testing Data
Test functions to ensure data is imported correctly and queries are providing expected results.

In [ ]:
SELECT *
FROM ..CovidDeaths
ORDER BY 2,3
 



In [ ]:
SELECT *
FROM ..CovidVaccinations
ORDER BY 2,3

## Total Cases vs. Total Deaths (Percentage of Deaths from Covid)

In [ ]:
-- Data Used for Death Percentages
Select cd.[location], cd.[date], cd.total_cases, cd.new_cases, cd.total_deaths, cd.population
FROM ..CovidDeaths cd
ORDER BY 1,2

In [ ]:
-- Calculating Percent of Deaths from Covid
Select cd.[location], cd.[date], cd.total_cases, cd.total_deaths, (cd.total_deaths * 100.0)/cd.total_cases AS DeathPercentage
FROM ..CovidDeaths cd
ORDER BY 1,2

In [ ]:
-- Deaths Percentage of U.S. Specifically
Select cd.[location], cd.[date], cd.total_cases, cd.total_deaths, (cd.total_deaths * 100.0)/cd.total_cases AS DeathPercentage
FROM ..CovidDeaths cd
WHERE cd.[location] like '%States'
ORDER BY 1,2

The Death percentage in the United States started off high due to the limited total cases reported. Around April of 2022, the percentage of deaths from individuals with Covid was at its highest with around a 6.29 percent and the percentage slowly began declining at the start of May in 2020.
As of August of 2022, the death percentage of individuals with Covid is abour a 1.11 percent. This can be due to precautions the United States has taken regarding mask mandates and social distancing, or the result of newly discovered medical treatment such as the COVID vaccination and booster shots available to the public.

## Infection Rate

In [ ]:
-- Total Cases vs. Population
Select cd.[location], cd.[date], cd.population,  cd.total_cases, (cd.total_cases * 100.0)/cd.population AS InfectedPopulation
FROM ..CovidDeaths cd
ORDER BY 1,2

Shows the growth in percent of total population infected every day for each country. From this data we can see when a country or countries exceeded certain metrics, such as how long it took for 5 percent of the country to become infected from Covid, or the length of time taken for a country to have 10 percent of their population infected to 15 percent.

In [ ]:
-- Countries with highest Infection Rate Compared to Population
Select cd.[location], cd.population, date,  MAX(cd.total_cases) AS HighestInfectionCount, MAX((cd.total_cases * 100.0)/cd.population) AS InfectedPopulation
FROM ..CovidDeaths cd
GROUP BY cd.[location], cd.population, [date]
ORDER BY InfectedPopulation DESC

Many small countries such as Faeroe Islands and Cuprus have the highest infection population, with more than 60 percent of the population being infected, but there ar elarger countries such as Portugal and France, which is populated by tens of millions, and have an infection percent at around 50 percent. The United States has about 27 percent of the population reported to have been infected. From this data conclusions can be drawn about a country's ability in preventing the spread of the virus. Since France is surrounded by small countries that are high in percent of population infected, France could have had difficulty preventing infected individuals from entered and exiting their country.

## Death Count

In [ ]:
-- Countries with Highest Death Count per Population
Select cd.[location],  MAX(cd.total_deaths) AS TotalDeathCount
FROM ..CovidDeaths cd
WHERE continent is not null
GROUP BY cd.[location]
ORDER BY TotalDeathCount DESC

In [ ]:
-- Continent With Highest Death Count
Select cd.[location],  MAX(cd.total_deaths) AS TotalDeathCount
FROM ..CovidDeaths cd
WHERE continent is null and iso_code NOT IN ('OWID_LIC', 'OWID_HIC','OWID_LMC','OWID_UMC' ) and [location] not in ('World', 'European Union', 'International')
GROUP BY cd.[location]
ORDER BY TotalDeathCount DESC

In [ ]:
---- Total Cases vs. Total Deaths GLOBALLY for every day
Select cd.[date], SUM(cd.new_cases) as total_cases, SUM(cd.new_deaths) as total_deaths, (SUM(cd.new_deaths) * 100.0)/SUM(cd.new_cases) AS DeathPercentage
FROM ..CovidDeaths cd
WHERE continent is not null
GROUP BY [date]
ORDER BY 1,2

In [ ]:
-- Total Death Percentage Across All Countries
Select  SUM(cd.new_cases) as total_cases, SUM(cd.new_deaths) as total_deaths, (SUM(cd.new_deaths) * 100.0)/SUM(cd.new_cases) AS DeathPercentage
FROM ..CovidDeaths cd
WHERE continent is not null
ORDER BY 1,2

Roughly 1.08 percent of the entire global population, of all pariticapting countries, had covid resulted in deaths. This is about 6.4 million people that lost their lvies after reporting they had Covid. This statistic can be used to show the deadly impact of the pandemic and can help promote the necessity of vaccines. Also can be used to show the significant need for all countries to work together during this unexpected time, to ensure the death count stays low and people all around the world are able to live safely.

## Joining Covid Deaths and Covid Vaccination Tables

In [ ]:
SELECT *
FROM ..CovidDeaths cd
JOIN ..CovidVaccinations cv
    ON cd.[date] = cv.[date]
    and cd.[location] = cv.[location]

## Vaccination Count

In [ ]:
-- Daily Count of New Covid Vaccinations
SELECT cd.[continent], cd.[location], cd.[date], cd.population, cv.new_vaccinations
FROM ..CovidDeaths cd
JOIN ..CovidVaccinations cv
    ON cd.[date] = cv.[date]
    and cd.[location] = cv.[location]
WHERE cd.continent is not NULL 
ORDER BY 2,3

Can be useful to track the spread of the vaccine for each country, and see which countries might need to increase efforts in spreading the vaccine among the public. Also tracks the vaccinations on a daily basis allowing for further investigation on what days of the week received the most vaccinations for a certain country, which can help countries decice which days they should be focusing more of their resources on, in order to assist as many people as they can.

In [ ]:
-- Rolling Count of Percent of Population that is vaccinated
With PopvsVac (Continent, Location, Date, Population, new_vaccinations, rolling_vaccinations)
as
(
SELECT cd.[continent], cd.[location], cd.[date], cd.population, cv.new_vaccinations
, SUM(CONVERT(bigint,cv.new_vaccinations)) OVER (PARTITION BY cd.LOCATION ORDER BY cd.location, cd.date) AS rolling_vaccinations
FROM ..CovidDeaths cd
JOIN ..CovidVaccinations cv
    ON cd.[location] = cv.[location]
    and cd.[date] = cv.[date]
WHERE cd.continent is not NULL

)
SELECT *, (rolling_vaccinations * 100.0/Population) As VaccinationPercentage
FROM PopvsVac

This table can help countries compare the total number of vaccined individuals and the entire population to see if they should dedicate more time and resources in spreading the vaccine. For example, we can see that it took the United States about 2 weeks to vaccindate 1 percent of their total population, which was about 363 thousand individuals. From then it took our country abour a month to break the 10 percent marker of total population vaccinated. This was over 34 million indiviudals. The table shows that within a month and a half, the country was able to provide over 34 million indiviudals with the highly demanded vaccine, showing that had a decent system considering the constraints that had to be met, such as deadlines, limited resources, and so on.

In [ ]:
-- Percentage of population Hospitalized
Select cd.[location], cd.[date], cd.total_cases, cd.hosp_patients, (cd.hosp_patients * 100.0)/cd.total_cases AS HospitalizedPercentage
FROM ..CovidDeaths cd
Where hosp_patients is not null
ORDER BY 1,2

The Covid Pandemic showed how unexpected situations can cause panic and need of medical assistance. The data gathered from COVID can be used to help establish how much/many resources and medical expertise may be needed in future pandemics.

In [ ]:
-- Countries with highest Death Count vs. Number of Patients Hospitalized
Select cd.[location], MAX(cd.total_deaths) AS TotalDeathCount, MAX(cd.hosp_patients) AS TotalHospitalized
FROM ..CovidDeaths cd
WHERE continent is not null
GROUP BY cd.[location]
ORDER BY TotalDeathCount DESC

The United States had the highest Death Count, but also had the highest Hopstalized Count. Could help show how the country was able to quicly adjust to the unique constraints and help as many individuals as they could.

## Creating Views to Store Data for Visualization

In [ ]:
-- -- Total Cases vs. Total Deaths
-- Create VIEW TotalCasesVsTotalDeaths as
-- Select cd.[location], cd.[date], cd.total_cases, cd.total_deaths, (cd.total_deaths * 100.0)/cd.total_cases AS DeathPercentage
-- FROM ..CovidDeaths cd

In [ ]:
-- -- Looking at Death Percentage of U.S. Specifically
-- Create View USDeathPercentage as
-- Select cd.[location], cd.[date], cd.total_cases, cd.total_deaths, (cd.total_deaths * 100.0)/cd.total_cases AS DeathPercentage
-- FROM ..CovidDeaths cd
-- WHERE location like '%States'


In [ ]:
-- -- Total Cases vs. Population
-- Create View TotalCasesVsPopulation as
-- Select cd.[location], cd.[date], cd.population,  cd.total_cases, (cd.total_cases * 100.0)/cd.population AS InfectedPopulation
-- FROM ..CovidDeaths cd


In [ ]:
-- -- Total Cases vs. Population for US Specifically
-- Create View USTotalCasesVsPopulation as
-- Select cd.[location], cd.[date], cd.population,  cd.total_cases, (cd.total_cases * 100.0)/cd.population AS InfectedPopulation
-- FROM ..CovidDeaths cd
-- WHERE location like '%States'

